In [57]:
from pysentimiento import create_analyzer
import os
import pandas as pd

In [58]:
analyzer = create_analyzer(task="sentiment", lang="en")
emotion_analyzer = create_analyzer(task="emotion", lang="en")

loading file https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/vocab.txt from cache at C:\Users\aupab/.cache\huggingface\transformers\973dbacfdf4c488622f01d1a226089e9e3dba130a0c3c11c2e36d49466fa40a8.f8a4dfe5c3c45a26f9df849d732decb191dc0c05ab270799695430332d143982
loading file https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/bpe.codes from cache at C:\Users\aupab/.cache\huggingface\transformers\0e474c44ff353f3b378fb140e7e6d4431df4ec6142e8b38d584c0dbc5afc3521.75877d86011e5d5d46614d3a21757b705e9d20ed45a019805d25159b4837b0a4
loading file https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/added_tokens.json from cache at C:\Users\aupab/.cache\huggingface\transformers\fe46927817477a58ec2aa92ef52f8ee6fc9e824d054f4aa6a3c129724dc9c9b7.99342f91bcc91195061d858c5ed9d0f61d2a8b6d7e3ef661b2a10fcacadecace
loading file https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/s

In [59]:
# set folder path
folder_path = "data/"

# get list of files in folder
file_list = os.listdir(folder_path)

# create empty list to hold dataframes
df_list = []

# loop through files in folder
for file in file_list:
    # check if file is a CSV
    if file.endswith('.csv'):
        # read CSV file into a pandas dataframe
        df = pd.read_csv(os.path.join(folder_path, file))
        # append dataframe to list
        df_list.append(df)

# concatenate all dataframes in list into a single dataframe
df = pd.concat(df_list, axis=0, ignore_index=True)

C:\Users\aupab\AppData\Local\Temp/ipykernel_35712/1325675326.py:15: DtypeWarning: Columns (60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
C:\Users\aupab\AppData\Local\Temp/ipykernel_35712/1325675326.py:15: DtypeWarning: Columns (60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
C:\Users\aupab\AppData\Local\Temp/ipykernel_35712/1325675326.py:15: DtypeWarning: Columns (58,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
C:\Users\aupab\AppData\Local\Temp/ipykernel_35712/1325675326.py:15: DtypeWarning: Columns (17,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))


In [60]:
print(df.columns[[17, 58,60, 62]])

Index(['is_eq_divided', 'advocates_4_id', 'advocates_5_id', 'advocates_6_id'], dtype='object')


In [61]:
df = pd.merge(df, df[['id', 'speaker_type']], left_on='reply_to', right_on='id', how='left').drop(['id_y'], axis=1)
df.rename(columns={'id_x': 'id', 'speaker_type_x': 'speaker_type', 'speaker_type_y': 'speaker_type_reply_to' }, inplace=True)


In [62]:
df = pd.merge(df, df[['id', 'speaker_type']], left_on='addressing', right_on='id', how='left').drop(['id_y'], axis=1)
df.rename(columns={'id_x': 'id', 'speaker_type_x': 'speaker_type', 'speaker_type_y': 'speaker_type_addressing'}, inplace=True)
print(df)

                  id                                               text  \
0       13186__0_000  I'm including my remarks with respect to the (...   
1       13186__0_001  You -- I suppose have put in your brief the na...   
2       13186__0_002  No, Your Honor. We haven't got the names of th...   
3       13186__0_003  If this is material, I should think, might be ...   
4       13186__0_004                           Oh, we could give you --   
...              ...                                                ...   
395540  24615__2_009  -- they can designate it and then a deliberate...   
395541  24615__2_010       Could I answer that question, Chief Justice?   
395542  24615__2_011                                           Briefly.   
395543  24615__2_012  I mean, you know, the government has made abso...   
395544  24615__2_013         Thank you, counsel. The case is submitted.   

                     speaker      reply_to  conversation_id     case_id  \
0             charles_k_

In [63]:
df['to_same_speaker'] = df['speaker_type_reply_to'] == df['speaker_type_addressing']
#If one is NaN Leave as True - we can still use it. 
df['to_same_speaker'] = df.apply(lambda x: True if pd.isna(x['speaker_type_reply_to']) or pd.isna(x['speaker_type_addressing']) else x['to_same_speaker'], axis=1)
print(df)


                  id                                               text  \
0       13186__0_000  I'm including my remarks with respect to the (...   
1       13186__0_001  You -- I suppose have put in your brief the na...   
2       13186__0_002  No, Your Honor. We haven't got the names of th...   
3       13186__0_003  If this is material, I should think, might be ...   
4       13186__0_004                           Oh, we could give you --   
...              ...                                                ...   
395540  24615__2_009  -- they can designate it and then a deliberate...   
395541  24615__2_010       Could I answer that question, Chief Justice?   
395542  24615__2_011                                           Briefly.   
395543  24615__2_012  I mean, you know, the government has made abso...   
395544  24615__2_013         Thank you, counsel. The case is submitted.   

                     speaker      reply_to  conversation_id     case_id  \
0             charles_k_

In [64]:
probs = analyzer.predict("GREAT DAY").probas
print(probs)
neg = probs['NEG']
pos = probs['POS']
neu = probs['NEU']


{'NEG': 0.0028708146419376135, 'NEU': 0.00686156190931797, 'POS': 0.9902676939964294}


In [65]:
a = emotion_analyzer.predict("good job Jonas").probas
a

{'others': 0.023973939940333366,
 'joy': 0.9668319225311279,
 'sadness': 0.0015657971380278468,
 'anger': 0.0028981217183172703,
 'surprise': 0.0019361898303031921,
 'disgust': 0.0014433461474254727,
 'fear': 0.001350751263089478}